In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import configparser as configparser
from ipywidgets import widgets

In [2]:
config = configparser.ConfigParser()
config.read('../config.ini')
mapbox_token = config['mapbox']['secret_token']

In [3]:
df = pd.read_csv('inno_geocoded.csv')
df = df.dropna(subset=['lng', 'lat'])

df['innoint'] = df.inno_nr.str.strip('inno_').astype('int')
df['date'] = pd.to_datetime(df['created_at']).dt.date

df.head(3)

/tmp/ipykernel_3069/1224049548.py:1: DtypeWarning: Columns (53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('inno_geocoded.csv')


,created_at,twitter_id,user_id,user_location,longitude,latitude,place_box_1_long,place_box_1_lat,place_box_2_long,place_box_2_lat,...,quarter,suburb,allotments,farm,hamlet,isolated_dwelling,island,time,innoint,date
0,2012-01-01 08:10:59+00:00,153387731521773569,191216844,Paraguay,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"01/01/2012, 08:10:59",2,2012-01-01
2,2012-01-03 14:15:36+00:00,154204266041585664,298356610,bogota,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"03/01/2012, 14:15:36",2,2012-01-03
3,2012-01-08 02:41:19+00:00,155841483243663360,285206171,Ibagué,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"08/01/2012, 02:41:19",2,2012-01-08


In [142]:
inno_choose = 'inno_02'
def process_geocodedtwitter_data(df, zone = 'country_y', inno_choose=inno_choose):

    # Choose Innovation
    df = df[df.inno_nr == inno_choose]
    df = df.reset_index()

    df= df.set_index(['date', 'twitter_id']) 

    df = df[['lat', 'lng']].copy()

    # Filling nan values with 0
    df = df.fillna(0)

    # Compute bubble sizes
    df['size'] = 1 #df[inno_choose]

    # Compute bubble color
    N = len(df)
    HSL_tuples = [f'hsl({int(x*350.0/N)}, 50%, 50%)' for x in range(N)] 
    df['color'] = HSL_tuples #df[cats].sum(axis=1)
    
    return df

In [143]:
df_viz = process_geocodedtwitter_data(df)
tester = df_viz.tail(5)

In [144]:
tester

lat        lng  size               color
date       twitter_id                                                         
2019-12-05 1202698183513714688 -31.761336 -71.318770     1  hsl(349, 50%, 50%)
           1202709487175507968  19.432630 -99.133178     1  hsl(349, 50%, 50%)
2019-12-06 1202758938015469568  39.466539  -0.332112     1  hsl(349, 50%, 50%)
           1203022667466596352  41.355724   2.070622     1  hsl(349, 50%, 50%)
2019-12-07 1203112131979333637   8.001871 -66.110932     1  hsl(349, 50%, 50%)

In [151]:
df_perm = pd.DataFrame()
for idx, data in tester.groupby(level=0):

    # build new multiindex model
    # add to previous
    # append


    print(tester.loc(idx))
    idprev = idx
    

ValueError: No axis named 2019-12-05 for object type DataFrame

In [111]:
# Selecting the day to display
#day = '2012-01-01'
#tmp = df#.xs(day)
tmp = df_viz

# Create the figure and feed it all the prepared columns
fig = go.Figure(
    go.Scattermapbox(
        lat=tmp['lat'],
        lon=tmp['lng'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=tmp['size']*10,
            color= tmp['color'],
            showscale=False,
            colorbar={'title':'Innovations', 'titleside':'top', 'thickness':4, 'tickprefix':' Inno Nr.: '}
        )
    )
)
# Specify layout information
fig.update_layout(
    mapbox=dict(
        accesstoken=mapbox_token,
        center=go.layout.mapbox.Center(lat=7, lon=-33),
        zoom=1,
        style="open-street-map"
    ),

)

# Display the figure
fig.show()

In [113]:
days = df_viz.index.levels[0].tolist()

In [133]:
frames = [{   
    'name':'frame_{}'.format(day),
    'data':[{
        'type':'scattermapbox',
        'lat':df_viz.xs(day)['lat'],
        'lon':df_viz.xs(day)['lng'],
        'marker':go.scattermapbox.Marker(
            size=df_viz.xs(day)['size'],
            color=df_viz.xs(day)['color'],
            showscale=False,
            colorbar={'title':'Innovations', 'titleside':'top', 'thickness':4, 'tickprefix':' Inno Nr.: '},
        )
    }],           
} for day in days]  


len(frames)

631

In [134]:
frames[1]

{'name': 'frame_2012-01-03',
 'data': [{'type': 'scattermapbox',
   'lat': twitter_id
   154204266041585664    4.653333
   Name: lat, dtype: float64,
   'lon': twitter_id
   154204266041585664   -74.083652
   Name: lng, dtype: float64,
   'marker': scattermapbox.Marker({
       'color': array(['hsl(0, 50%, 50%)'], dtype=object),
       'colorbar': {'thickness': 4, 'tickprefix': ' Inno Nr.: ', 'title': {'side': 'top', 'text': 'Innovations'}},
       'showscale': False,
       'size': array([1])
   })}]}

In [130]:
frames = []
for day in days:
    frame = {'data': [], 'name': 'frame_{}'.format(day)}
    data_dict = {
        'type':'scattermapbox',
        'lat':df_viz.xs(day)['lat'],
        'lon':df_viz.xs(day)['lng'],
        'marker':go.scattermapbox.Marker(
            size=df_viz.xs(day)['size'],
            color=df_viz.xs(day)['color'],
            showscale=False,
            colorbar={'title':'Innovations', 'titleside':'top', 'thickness':4, 'tickprefix':' Inno Nr.: '},
        )
    },
   
    frame['data'].append(data_dict)
    frame = frames.append(frame)
    frames.append(frame)

len(frames)

1262

In [115]:
sliders = [{
    'transition':{'duration': 0},
    'x':0.08, 
    'len':0.88,
    'currentvalue':{'font':{'size':15}, 'prefix':'📅 ', 'visible':True, 'xanchor':'center'},  
    'steps':[{'label':str(day),
            'method':'animate',
            'args':[
                ['frame_{}'.format(day)],
                {'mode':'immediate', 'frame':{'duration':100, 'redraw': True}, 'transition':{'duration':50}}
              ],
        } for day in days]
}]



In [116]:
play_button = [{
    'type':'buttons',
    'showactive':True,
    'x':0.045, 'y':-0.08,
    'buttons':[{ 
        'label':'🎬', # Play
        'method':'animate',
        'args':[
            None,
            {
                'frame':{'duration':100, 'redraw':True},
                'transition':{'duration':50},
                'fromcurrent':True,
                'mode':'immediate',
            }
        ]
    }]
}]

In [132]:
# Defining the initial state
data = frames[0]['data']

# Adding all sliders and play button to the layout
layout = go.Layout(
    sliders=sliders,
    updatemenus=play_button,
    mapbox={
        'accesstoken':mapbox_token,
        'center':{'lat':7, 'lon':-33},
        'zoom':1.7,
        'style':"open-street-map",
        
    }
)

# Creating the figure
fig = go.Figure(data=data, layout=layout, frames=frames)

# Displaying the figure
fig.show()

ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [({'type': 'scattermapbox', 'lat': twitter_id
153387731521773569   -23.316593
Name: lat, dtype: float64, 'lon': twitter_id
153387731521773569   -58.169345
Name: lng, dtype: float64, 'marker': scattermapbox.Marker({
    'color': array(['hsl(0, 50%, 50%)'], dtype=object),
    'colorbar': {'thickness': 4, 'tickprefix': ' Inno Nr.: ', 'title': {'side': 'top', 'text': 'Innovations'}},
    'showscale': False,
    'size': array([1])
})},)]

    The 'data' property is a tuple of trace instances
    that may be specified as:
      - A list or tuple of trace instances
        (e.g. [Scatter(...), Bar(...)])
      - A single trace instance
        (e.g. Scatter(...), Bar(...), etc.)
      - A list or tuple of dicts of string/value properties where:
        - The 'type' property specifies the trace type
            One of: ['bar', 'barpolar', 'box', 'candlestick',
                     'carpet

# Hist

In [99]:
df = df.reset_index()

In [100]:
import plotly.express as px

fig = px.histogram(df, x='date', color="inno_nr", title='Temporal Distribution of Innovations')
fig.update_layout(bargap=0.2)
fig.show()

In [101]:
df_f = df_viz.reset_index()
fig = px.histogram(df_f, x='date', color='country_y', title=f'Geo-Temporal Distribution of {inno_choose}')
fig.update_layout(bargap=0.2)
fig.show()